<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/CopaTrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import ta # Biblioteca de Análise Técnica

# --- Configuração dos Ativos ---
configuracao_ativos = {
    "Mini Índice":    {"ticker": "^BVSP", "max_contratos": 10},
    "Mini Dólar":     {"ticker": "BRL=X", "max_contratos": 5},
    "Bitcoin Futuro": {"ticker": "BTC-USD", "max_contratos": 20},
    "Ether Futuro":   {"ticker": "ETH-USD", "max_contratos": 10},
    "Solana Futuro":  {"ticker": "SOL-USD", "max_contratos": 10},
    "Ouro Futuro":    {"ticker": "GC=F", "max_contratos": 5},
    "Euro Futuro":    {"ticker": "EURBRL=X", "max_contratos": 5}
}

# --- Função de Análise Enriquecida ---
def analisar_ativo(ticker):
    """
    Busca os dados de um ativo, calcula múltiplos indicadores (MM, RSI, MACD)
    e retorna uma recomendação baseada na confluência de sinais.
    """
    try:
        dados = yf.download(ticker, period="90d", interval="1d", progress=False, auto_adjust=True)

        if dados.empty or len(dados) < 30:
            return {"Preço": "N/A", "RSI": "N/A", "Recomendação": "Dados insuficientes"}

        # Garante que os dados de fechamento são 1-dimensionais
        close_prices = dados['Close'].squeeze() # <-- CORREÇÃO PRINCIPAL APLICADA AQUI

        # --- Indicadores Base ---
        ultimo_preco = close_prices.iloc[-1]
        mm_curta = close_prices.ewm(span=9, adjust=False).mean().iloc[-1]
        mm_longa = close_prices.ewm(span=21, adjust=False).mean().iloc[-1]

        # Usamos a variável 'close_prices' já corrigida
        rsi = ta.momentum.RSIIndicator(close_prices, window=14).rsi().iloc[-1]

        # --- NOVO INDICADOR: MACD ---
        # Usamos a variável 'close_prices' já corrigida
        macd_indicador = ta.trend.MACD(close_prices)
        macd_linha = macd_indicador.macd().iloc[-1]
        macd_sinal = macd_indicador.macd_signal().iloc[-1]

        # --- LÓGICA DE RECOMENDAÇÃO (CONFLUÊNCIA) ---
        recomendacao = "NEUTRO / MONITORAR"

        # Cenário 1: COMPRA FORTE
        if (mm_curta > mm_longa) and (macd_linha > macd_sinal) and (rsi > 50 and rsi < 70):
            recomendacao = "COMPRA FORTE (Tendência e Momentum de Alta)"

        # Cenário 2: VENDA FORTE
        elif (mm_curta < mm_longa) and (macd_linha < macd_sinal) and (rsi < 50 and rsi > 30):
            recomendacao = "VENDA FORTE (Tendência e Momentum de Baixa)"

        # Cenário 3: POSSÍVEL COMPRA POR REVERSÃO
        elif (rsi < 30) and (macd_linha > macd_sinal):
            recomendacao = "MONITORAR COMPRA (Possível Reversão de Baixa)"

        # Cenário 4: POSSÍVEL VENDA POR REVERSÃO
        elif (rsi > 70) and (macd_linha < macd_sinal):
            recomendacao = "MONITORAR VENDA (Possível Reversão de Alta)"

        return {
            "Preço": f"{ultimo_preco:.2f}",
            "RSI": f"{rsi:.2f}" if isinstance(rsi, (int, float)) else "N/A",
            "Recomendação": recomendacao
        }
    except Exception as e:
        return {"Preço": "N/A", "RSI": "N/A", "Recomendação": f"ERRO: {str(e)}"}

# --- Roda a Análise e Exibe o Painel ---
if __name__ == "__main__":
    print("=========================================================")
    print("      ANALISTA DE ATIVOS - COPA TRADE BTG (V5 Corrigida)")
    print(f"      Análise gerada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print("=========================================================")

    resultados = []
    for nome, config in configuracao_ativos.items():
        print(f"-> Analisando {nome} ({config['ticker']})...")
        analise = analisar_ativo(config['ticker'])
        resultados.append({
            "Ativo": nome,
            "Preço Atual": analise["Preço"],
            "RSI(14)": analise["RSI"],
            "Recomendação de Ação": analise["Recomendação"]
        })

    painel = pd.DataFrame(resultados)

    print("\n--- PAINEL DE ANÁLISE TÉCNICA AVANÇADA ---")
    pd.set_option('display.max_colwidth', None)
    print(painel.to_string(index=False))
    print("\nLembre-se: Esta é uma ferramenta de apoio. Opere com consciência e gerencie seus riscos.")

      ANALISTA DE ATIVOS - COPA TRADE BTG (V5 Corrigida)
      Análise gerada em: 06/10/2025 22:47:32
-> Analisando Mini Índice (^BVSP)...
-> Analisando Mini Dólar (BRL=X)...
-> Analisando Bitcoin Futuro (BTC-USD)...
-> Analisando Ether Futuro (ETH-USD)...
-> Analisando Solana Futuro (SOL-USD)...
-> Analisando Ouro Futuro (GC=F)...
-> Analisando Euro Futuro (EURBRL=X)...

--- PAINEL DE ANÁLISE TÉCNICA AVANÇADA ---
         Ativo Preço Atual RSI(14)                        Recomendação de Ação
   Mini Índice   143608.08   51.12                          NEUTRO / MONITORAR
    Mini Dólar        5.31   40.73                          NEUTRO / MONITORAR
Bitcoin Futuro   124891.78   72.95                          NEUTRO / MONITORAR
  Ether Futuro     4687.78   63.27 COMPRA FORTE (Tendência e Momentum de Alta)
 Solana Futuro      234.41   58.16 COMPRA FORTE (Tendência e Momentum de Alta)
   Ouro Futuro     3984.40   81.01                          NEUTRO / MONITORAR
   Euro Futuro        6.22   